<a href="https://colab.research.google.com/github/dilshod-obidov/yolo_tracking/blob/master/yolo_tracking_ui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. app opens
2. choose tracking method
3. run button



# Setup

Clone repo, install dependencies and check PyTorch and GPU.

In [1]:
!pip install --upgrade pip setuptools wheel
!git clone https://github.com/dilshod-obidov/yolo_tracking  # clone repo
%cd yolo_tracking
!pip install -e .

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 2.2.1+cu121 (CPU)


# Streamlit Setup

In [2]:
!pip install streamlit
!apt-get install ffmpeg
clear_output()

In [3]:
%cd /content/yolo_tracking/tracking

/content/yolo_tracking/tracking


## Streamlit app

In [62]:
%%writefile app.py
import streamlit as st
import os
from tempfile import NamedTemporaryFile

# Define the tracking methods
TRACKING_METHODS = ['bytetrack', 'botsort', 'strongsort', 'ocsort', 'deepocsort', 'hybridsort']

st.title("YOLO Tracking Method Selector")

# Dropdown menu for choosing the tracking method
tracking_method = st.selectbox(
    "Choose Tracking Method:",
    TRACKING_METHODS
)

# File uploader for the video file
uploaded_file = st.file_uploader("Choose a video file", type=["mp4", "avi", "mov"])

# Run button
if st.button("Run"):
    if uploaded_file is not None:
        # Save the uploaded video file to a temporary location
        with NamedTemporaryFile(delete=False) as temp_file:
            temp_file.write(uploaded_file.read())
            temp_filename = temp_file.name

        st.write(f"Running {tracking_method} tracking on the video...")

        # Save the uploaded video as .mp4 format
        file_path = '/content/uploaded_video.mp4'  # Save as .mp4
        with open(file_path, 'wb') as f:
            f.write(uploaded_file.getvalue())

        # Run the tracking command
        command = f'python track.py --tracking-method {tracking_method} --source {file_path} --save --project /content --name processed_video --exist-ok'
        os.system(command)

        # Convert the processed video from .avi to .mp4
        os.system('ffmpeg -y -i /content/processed_video/uploaded_video.avi /content/processed_video/processed_video.mp4')

        st.write("Processing completed.")
    else:
        st.write("Please upload a video file.")

    # Path to the processed video
    processed_video_path = '/content/processed_video/processed_video.mp4'

    # Display the processed video
    if os.path.exists(processed_video_path):
        st.title("Processed Video Player")
        with open(processed_video_path, 'rb') as video_file:
            video_bytes = video_file.read()
        st.video(video_bytes)
    else:
        st.write("Processed video not found. Please run the tracking first.")


Overwriting app.py


In [20]:
! wget -q -O - ipv4.icanhazip.com

35.201.165.130


In [63]:
! streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.201.165.130:8501

npx: installed 22 in 1.785s
your url is: https://tame-hounds-teach.loca.lt

video 1/1 (1/150) /content/uploaded_video.mp4: 384x640 12 persons, 1 car, 1 motorcycle, 176.4ms
video 1/1 (2/150) /content/uploaded_video.mp4: 384x640 12 persons, 1 car, 1 motorcycle, 199.7ms
video 1/1 (3/150) /content/uploaded_video.mp4: 384x640 12 persons, 1 car, 1 motorcycle, 202.7ms
video 1/1 (4/150) /content/uploaded_video.mp4: 384x640 12 persons, 1 car, 1 motorcycle, 210.2ms
video 1/1 (5/150) /content/uploaded_video.mp4: 384x640 12 persons, 1 car, 1 motorcycle, 201.3ms
video 1/1 (6/150) /content/uploaded_video.mp4: 384x640 12 persons, 1 car, 1 motorcycle, 191.2ms
video 1/1 (7/150) /content/uploaded_video.mp4: 384x640 12 persons, 1 car, 1 motorcycle, 206.8ms
video 1/1 (8/150) /content/uploaded_video.mp4: 384x640 13 persons, 1 car, 1 motorcycle, 203.0ms
video 1/1 (9/1

# Download data

Get test video from repo and extract the first 2 seconds of it

In [ ]:
# get the test video from the repo
!wget -nc https://github.com/mikel-brostrom/yolo_tracking/releases/download/v.2.0/test.avi
# extract 3 seconds worth of video frames of it
!yes | ffmpeg -ss 00:00:00 -i test.avi -t 00:00:01 -vf fps=30 out.avi

File ‘test.avi’ already there; not retrieving.

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libv

## Run inference on video

The ``cv2.imshow()`` and ``cv.imshow()`` functions from the [opencv-python](https://github.com/skvark/opencv-python) package are incompatible with Jupyter notebook; see https://github.com/jupyter/notebook/issues/3935.

Hence we chose to save it to file in this notebook. Locally you can use the ``--show-vid`` flag in order visualize the tracking in real-time

In [ ]:
!python tracking/track.py --yolo-model yolov8n.pt --reid-model osnet_x0_25_msmt17.pt --source out.avi --save --conf 0.2

2024-05-16 13:13:45.991 | ERROR    | boxmot.utils.checks:check_packages:26 - The 'ultralytics@ git+https://github.com/mikel-brostrom/ultralytics.git' distribution was not found and is required by the application
2024-05-16 13:13:45.992 | WARNING  | boxmot.utils.checks:check_packages:29 - 
Missing packages: "ultralytics @ git+https://github.com/mikel-brostrom/ultralytics.git" 
Atempting installation...
2024-05-16 13:13:53.908 | SUCCESS  | boxmot.utils.checks:check_packages:35 - All the missing packages were installed successfully
100% 6.23M/6.23M [00:00<00:00, 78.1MB/s]

2024-05-16 13:13:56.465 | INFO     | boxmot.utils.torch_utils:select_device:52 - Yolo Tracking v10.0.72 🚀 Python-3.10.12 torch-2.2.1+cu121CPU
Downloading...
From: https://drive.google.com/uc?id=1sSwXSUlj4_tHZequ_iZ8w_Jh0VaRQMqF
To: /content/yolo_tracking/osnet_x0_25_msmt17.pt
100% 3.06M/3.06M [00:00<00:00, 94.6MB/s]
2024-05-16 13:13:58.205 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:207 - S

# Show results

https://stackoverflow.com/questions/60977179/how-to-play-avi-file-in-google-colab

https://stackoverflow.com/questions/57377185/how-play-mp4-video-in-google-colab

Compress the video file to a format supported by Google Colab (mpeg4 (native) -> h264 (libx264))

In [ ]:
!ffmpeg -i /content/yolo_tracking/runs/track/exp/out.avi -vf fps=30 -vcodec libx264 output.mp4

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

Get the file content into data_url

In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('output.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

Display it with HTML

In [ ]:
HTML("""
<video controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

# Evaluate

In [ ]:
!python examples/val.py --tracking-method deepocsort --benchmark MOT17-mini --imgsz 320 --conf 0.2

2023-08-08 14:54:47.907915: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-08 14:54:49.446047: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
val: yolo_model=/content/yolo_tracking/examples/weights/yolov8n.pt, reid_model=/content/yolo_tracking/examples/weights/osnet_x0_25_msmt17.pt, tracking_method=deepocsort, name=exp, classes=['0'], project=/content/yolo_tracking/runs/val, exist_ok=False, gsi=False, benchmark=MOT17-mini, split=train, eval_existing=False, conf=0.2, imgsz=[320], device=[''], save=False, processes_per_device=2
2023-08-08 14:54:54.516 | INFO     | __main__:download_mot_eval_tools:63 - Official MOT evaluation repo downloaded
2023-08-08 14:54:54.517 | INFO     | __main__:eval:195 - St